In [1]:
# Load environment variables from .env file
from dotenv import load_dotenv

# Load .env file
load_dotenv()

import os
from vexa_client import VexaClient

import os
TOKEN = os.getenv('ADMIN_API_TOKEN')

In [2]:
admin_client = VexaClient(
    base_url="https://gateway.dev.vexa.ai",
    admin_key=TOKEN  # From docker-compose.yml
)

In [3]:
import random

In [4]:
new_user = admin_client.create_user(email=f"{random.randint(1, 1000000)}@example.com", name="test")



DEBUG: Making POST request to https://gateway.dev.vexa.ai/admin/users
DEBUG: Headers: {'Content-Type': 'application/json', 'X-Admin-API-Key': 'KJnl2jjk&'}
DEBUG: Params: None
DEBUG: JSON data: {'email': '264087@example.com', 'name': 'test'}
DEBUG: Response status: 201
DEBUG: Response headers: {'Server': 'nginx/1.18.0 (Ubuntu)', 'Date': 'Tue, 15 Apr 2025 15:57:30 GMT', 'Content-Type': 'application/json', 'Content-Length': '111', 'Connection': 'keep-alive'}
DEBUG: Response content: {"email":"264087@example.com","name":"test","image_url":null,"id":41,"created_at":"2025-04-15T15:57:30.103522"}...


In [3]:

user_id = new_user['id']

NameError: name 'new_user' is not defined

In [6]:
token_info = admin_client.create_token(user_id=user_id)
user_api_key = token_info['token']


DEBUG: Making POST request to https://gateway.dev.vexa.ai/admin/users/41/tokens
DEBUG: Headers: {'Content-Type': 'application/json', 'X-Admin-API-Key': 'KJnl2jjk&'}
DEBUG: Params: None
DEBUG: JSON data: None
DEBUG: Response status: 201
DEBUG: Response headers: {'Server': 'nginx/1.18.0 (Ubuntu)', 'Date': 'Tue, 15 Apr 2025 15:57:30 GMT', 'Content-Type': 'application/json', 'Content-Length': '115', 'Connection': 'keep-alive'}
DEBUG: Response content: {"user_id":41,"id":42,"token":"N8BP02ZFI5GVp8xt0GQV1cdSww44dsJAQHiII2Ws","created_at":"2025-04-15T15:57:30.268364"}...


In [7]:
user_api_key

'N8BP02ZFI5GVp8xt0GQV1cdSww44dsJAQHiII2Ws'

In [4]:
user_api_key = 'N8BP02ZFI5GVp8xt0GQV1cdSww44dsJAQHiII2Ws'

In [5]:
client = VexaClient(
    base_url="https://gateway.dev.vexa.ai",
    api_key=user_api_key
)

In [6]:
meeting_url="https://meet.google.com/zcx-poey-yjd"
native_meeting_id = meeting_url.split("/")[-1]  # "xyz-abcd-123"

In [10]:
meeting_info = client.request_bot(
    platform="google_meet",
    native_meeting_id=native_meeting_id,
    bot_name="MyBot"
)
meeting_id = meeting_info['id']


DEBUG: Making POST request to https://gateway.dev.vexa.ai/bots
DEBUG: Headers: {'Content-Type': 'application/json', 'X-API-Key': 'N8BP02ZFI5GVp8xt0GQV1cdSww44dsJAQHiII2Ws'}
DEBUG: Params: None
DEBUG: JSON data: {'platform': 'google_meet', 'native_meeting_id': 'zcx-poey-yjd', 'bot_name': 'MyBot'}
DEBUG: Response status: 503
DEBUG: Response headers: {'Server': 'nginx/1.18.0 (Ubuntu)', 'Date': 'Tue, 15 Apr 2025 15:57:35 GMT', 'Content-Type': 'application/json', 'Content-Length': '34', 'Connection': 'keep-alive'}
DEBUG: Response content: {"detail":"Service unavailable: "}...


VexaClientError: HTTP Error 503 for POST https://gateway.dev.vexa.ai/bots: Service unavailable: 

In [7]:
meetings = client.get_meetings()


DEBUG: Making GET request to https://gateway.dev.vexa.ai/meetings
DEBUG: Headers: {'Content-Type': 'application/json', 'X-API-Key': 'N8BP02ZFI5GVp8xt0GQV1cdSww44dsJAQHiII2Ws'}
DEBUG: Params: None
DEBUG: JSON data: None
DEBUG: Response status: 200
DEBUG: Response headers: {'Server': 'nginx/1.18.0 (Ubuntu)', 'Date': 'Tue, 15 Apr 2025 16:49:08 GMT', 'Content-Type': 'application/json', 'Content-Length': '408', 'Connection': 'keep-alive'}
DEBUG: Response content: {"meetings":[{"id":38,"user_id":41,"platform":"google_meet","native_meeting_id":"zcx-poey-yjd","constructed_meeting_url":"https://meet.google.com/zcx-poey-yjd","status":"active","bot_container_id":"28f025bf8351b0a0511635f22e9a596d28ff4d0176deaa4561199edc184704b1","start_time":"2025-04-15T15:57:41.868252","end_time":null,"created_at":"2025-04-15T15:57:30.671178","updated_at":"2025-04-15T15:57:30.713401"}]}...


In [8]:
meeting_id

NameError: name 'meeting_id' is not defined

In [9]:
import pandas as pd

In [10]:
# pd.options.display.max_colwidth = 1000
# pd.set_option('display.max_colwidth', 1000)
# pd.set_option('display.max_rows', 1000)
# pd.set_option('display.max_columns', 1000)
# pd.set_option('display.width', 1000)
# pd.set_option('display.float_format', '{:20.2f}'.format)
# pd.set_option('display.max_colwidth', 1000)
# pd.set_option('display.max_rows', 1000)
# pd.set_option('display.max_columns', 1000)

In [440]:

transcript = client.get_transcript(native_meeting_id=native_meeting_id,platform='google_meet')
segments = transcript.get('segments', [])
df = pd.DataFrame(transcript['segments']).sort_values(['created_at','start']).drop_duplicates('start',keep='last')
df.tail(5)


DEBUG: Making GET request to https://gateway.dev.vexa.ai/transcripts/google_meet/zcx-poey-yjd
DEBUG: Headers: {'Content-Type': 'application/json', 'X-API-Key': 'N8BP02ZFI5GVp8xt0GQV1cdSww44dsJAQHiII2Ws'}
DEBUG: Params: None
DEBUG: JSON data: None
DEBUG: Response status: 200
DEBUG: Response headers: {'Server': 'nginx/1.18.0 (Ubuntu)', 'Date': 'Tue, 15 Apr 2025 17:02:44 GMT', 'Content-Type': 'application/json', 'Content-Length': '91882', 'Connection': 'keep-alive'}
DEBUG: Response content: {"id":38,"platform":"google_meet","native_meeting_id":"zcx-poey-yjd","constructed_meeting_url":"https://meet.google.com/zcx-poey-yjd","status":"active","start_time":"2025-04-15T15:57:41.868252","end_time":null,"segments":[{"start":0.0,"end":3.994,"text":" и 23456789101240","language":null,"created_at":"2025-04-15T15:58:08.179649","speaker":null},{"start":0.0,"end":6.0,"text":" и 23456789101248","language":null,"created_at":"2025-04-15T15:58:10.376463","speaker":null},{"start":0.0,"end":5.572,"te...


,start,end,text,language,created_at,speaker
221,247.899,251.449,it really works,None,2025-04-15T17:02:32.361618,None
223,251.449,252.449,Thank you for joining us.,None,2025-04-15T17:02:33.776147,None
224,252.449,253.449,We'll see you next time.,None,2025-04-15T17:02:34.907096,None
225,253.449,253.469,Bye.,None,2025-04-15T17:02:34.907306,None
234,260.143,264.543,"It really works, and all sorts of shit doesn'...",None,2025-04-15T17:02:41.831872,None


In [ ]:
df['text'].tolist()

[" Yeah, so tell me about about yourself. What's that that you're doing?",
 ' Is it about...',
 ' Yeah',
 " There's a bit of a risk.",
 ' Okay?',
 " Yeah, I've been switching another, the proprietary version now, the other, the other.",
 ' the other',
 ' Thank you very much.',
 " You're most welcome.",
 " I'm just curious what types of model I've been using for that, I suppose it's all because",
 " I suppose it's a little bit...",
 " about yourself? What's that you're doing? Are you about, you said you're about machine learning?",
 " I suppose it's a lot of kind of boosting trees, yes, this is like a pretty large language",
 ' Is it about STG and TGS things, have you played around this stuff?',
 ' model. Yeah, so mostly table type of table data, right?',
 ' So mostly table type of table data, right?',
 ' See, okay.',
 ' Thank you very much.',
 ' Yes, yes.',
 ' Yeah.',
 ' Thanks.',
 ' Thank you.',
 " Yeah, I've been switching another the proprietary policy now the other",
 " Okay. Okay.

In [25]:
platform = 'google_meet'

In [ ]:

    
def get_transcript_segments(native_meeting_id, platform):
    
    
    meeting_id = conn.execute(
        select([metadata.tables['meetings'].c.id]).where(
            and_(
                metadata.tables['meetings'].c.native_meeting_id == native_meeting_id,
                metadata.tables['meetings'].c.platform == platform
            )
        )
    ).scalar()
    
    if not meeting_id:
        conn.close()
        return []
    
    result = conn.execute(
        select([metadata.tables['transcript_segments']]).where(
            metadata.tables['transcript_segments'].c.meeting_id == meeting_id
        ).order_by(metadata.tables['transcript_segments'].c.start_time)
    ).fetchall()
    
    conn.close()
    return result

if __name__ == "__main__":
    print(len(get_transcript_segments("nav-yijy-spp", "google_meet"))) 

NameError: name 'conn' is not defined

In [ ]:
"""Vexa API Client - Minimal Usage Snippets"""
from vexa_client import VexaClient

# --- Client Initialization ---
# User client
client = VexaClient(
    base_url="http://localhost:8056",
    api_key="your-user-api-key"
)

# Admin client 
admin_client = VexaClient(
    base_url="http://localhost:8056",
    admin_key="supersecretadmintoken"  # From docker-compose.yml
)

# --- Admin Operations ---
# Create user
new_user = admin_client.create_user(email="user@example.com", name="New User")
user_id = new_user['id']